# Comprehensive Test Suite - Ontario Damages Compendium

This notebook validates the injury-focused semantic search architecture:

1. **Camelot Table Extraction** - Hybrid lattice/stream approach
2. **Injury-Focused Embeddings** - Semantic similarity on injuries only
3. **Exclusive Region Filtering** - Multi-region case handling
4. **Meta-Score Computation** - Injury overlap, age/gender matching
5. **End-to-End Search Pipeline** - Integration tests with real queries
6. **Performance Benchmarks** - Speed and accuracy metrics

---

## Setup and Dependencies

In [ ]:
# Standard imports
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict
import time

# Camelot for table extraction (hybrid approach)
import camelot

# Embedding model
from sentence_transformers import SentenceTransformer

# App modules
from app.core.search import (
    search_cases,
    compute_meta_score,
    _injury_overlap_score,
    _age_proximity_score,
    _gender_match_score
)
from app.core.data_loader import initialize_data

print("✅ All imports successful")

---

## Test 1: Camelot Table Extraction

Verify that Camelot correctly extracts damage award tables from the PDF using hybrid lattice/stream approach.

In [ ]:
# Path to compendium PDF
PDF_PATH = "2024damagescompendium.pdf"  # Update with actual filename

# Text cleaning helper function (for DISPLAY only, not parsing)
import re

def clean_text_for_display(text):
    """Clean text for display by replacing escape characters and normalizing whitespace."""
    if not text or text == 'nan':
        return ""
    
    text = str(text)
    text = text.replace('\\n', ' ')  # Replace literal \n
    text = text.replace('\n', ' ')   # Replace actual newlines  
    text = re.sub(r'\s+', ' ', text) # Multiple spaces to single
    return text.strip()

def extract_section_from_stream(pdf_path, page_range):
    """Extract section header from stream mode row 0."""
    # Known anatomical section keywords
    section_keywords = {
        'General', 'Cervical Spine', 'Thoracic Spine', 'Lumbar Spine',
        'Shoulder', 'Elbow', 'Forearm', 'Wrist', 'Hand', 'Finger',
        'Hip', 'Knee', 'Lower Leg', 'Ankle', 'Foot', 'Toe',
        'Brain', 'Head', 'Face', 'Eye', 'Ear', 'Nose',
        'Psychological', 'Chronic Pain', 'Multiple Injuries'
    }
    
    try:
        # Use stream mode to capture section header
        tables_stream = camelot.read_pdf(pdf_path, pages=page_range, flavor="stream")
        
        if len(tables_stream) > 0:
            df_stream = tables_stream[0].df
            if len(df_stream) > 0:
                # Check row 0 for section keywords
                row0_values = df_stream.iloc[0].tolist()
                for cell in row0_values:
                    cell_str = str(cell).strip()
                    if cell_str in section_keywords:
                        return cell_str
        
        return None
    except:
        return None

# Test lattice mode (for bordered tables)
print("📊 Testing Camelot HYBRID mode (stream for section, lattice for data)...\n")

# Extract section using stream mode
page_range = "1-5"
section_from_stream = extract_section_from_stream(PDF_PATH, page_range)

# Extract table data using lattice mode
tables_lattice = camelot.read_pdf(PDF_PATH, pages=page_range, flavor="lattice")

print(f"✅ Lattice mode found {len(tables_lattice)} tables in pages {page_range}")
if section_from_stream:
    print(f"✅ Stream mode found section header: '{section_from_stream}'")
else:
    print(f"⚠️  Stream mode did not find section header in row 0")

if len(tables_lattice) > 0:
    # Get first table
    table = tables_lattice[0]
    df = table.df
    
    print(f"\n📋 First table analysis:")
    print(f"   Raw table shape: {df.shape}")
    print(f"   Parsing accuracy: {table.accuracy:.2f}%")
    print(f"   Page: {table.page}\n")
    
    # Parse table structure properly
    print("="*80)
    
    # IMPROVED STRUCTURE DETECTION
    # Check row 0 to determine table type:
    # 1. Multiple non-empty cells across row 0 → Headers spread across columns
    # 2. Only first cell filled, has newlines → Newline-separated headers
    # 3. Only first cell filled, no newlines → Section header
    
    row0_cell0 = str(df.iloc[0, 0]).strip() if len(df) > 0 else ""
    row0_values = [str(cell).strip() for cell in df.iloc[0].tolist()] if len(df) > 0 else []
    num_filled_cells = sum(1 for v in row0_values if v and v != 'nan')
    
    if num_filled_cells > 1:
        # Case 1: Headers spread across columns (like pages 91-95)
        column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row0_values)]
        # Use section from stream mode if available
        section_header = section_from_stream if section_from_stream else "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers spread across row 0 (multiple columns)")
        if section_from_stream:
            print(f"📚 SECTION (from stream mode): '{section_header}'")
        else:
            print(f"⚠️  No section found in stream mode row 0")
        print(f"\n📋 Column Headers (from row 0, spread across columns):")
    elif '\n' in row0_cell0 or '\\n' in row0_cell0:
        # Case 2: Newline-separated headers in first cell
        headers_raw = row0_cell0.replace('\\n', '\n').split('\n')
        column_headers = [h.strip() for h in headers_raw if h.strip()]
        section_header = section_from_stream if section_from_stream else "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers in row 0, col 0 (newline-separated)")
        if section_from_stream:
            print(f"📚 SECTION (from stream mode): '{section_header}'")
        print(f"\n📋 Column Headers (extracted from row 0, col 0):")
    else:
        # Case 3: Section header in row 0, headers in row 1
        section_header = row0_cell0
        print(f"📚 SECTION HEADER (from row 0): '{section_header}'")
        
        if len(df) > 1:
            row1_cell0 = str(df.iloc[1, 0]).strip()
            row1_values = [str(cell).strip() for cell in df.iloc[1].tolist()]
            num_filled_row1 = sum(1 for v in row1_values if v and v != 'nan')
            
            if '\n' in row1_cell0 or '\\n' in row1_cell0:
                # Headers newline-separated in row 1
                headers_raw = row1_cell0.replace('\\n', '\n').split('\n')
                column_headers = [h.strip() for h in headers_raw if h.strip()]
                print(f"\n📋 COLUMN HEADERS (extracted from row 1, col 0 - newline separated):")
            elif num_filled_row1 > 1:
                # Headers spread across row 1
                column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row1_values)]
                print(f"\n📋 COLUMN HEADERS (from row 1, spread across columns):")
            else:
                column_headers = [str(h).strip() for h in df.iloc[1].tolist() if str(h).strip()]
                print(f"\n📋 COLUMN HEADERS (from row 1):")
            
            data_start_row = 2
        else:
            column_headers = []
            data_start_row = 1
    
    # Display headers (cleaned for readability)
    for i, header in enumerate(column_headers):
        print(f"   {i}: {clean_text_for_display(header)}")
    
    # Data rows
    print(f"\n📊 Data Preview (Rows {data_start_row}-{data_start_row+2}, cleaned for display):")
    print("-"*80)
    
    if len(df) > data_start_row:
        # Create properly structured dataframe
        df_data = df.iloc[data_start_row:].copy()
        
        # Apply column headers
        if len(column_headers) == df_data.shape[1]:
            df_data.columns = column_headers
        else:
            # Pad or trim headers to match data columns
            if len(column_headers) < df_data.shape[1]:
                column_headers.extend([f"Col_{i}" for i in range(len(column_headers), df_data.shape[1])])
            else:
                column_headers = column_headers[:df_data.shape[1]]
            df_data.columns = column_headers
        
        # Clean data cells for DISPLAY only
        df_display = df_data.head(3).copy()
        for col in df_display.columns:
            df_display[col] = df_display[col].apply(lambda x: clean_text_for_display(str(x)))
        
        # Show first 3 data rows
        print(df_display.to_string())
        
        print(f"\n✅ Table properly parsed:")
        print(f"   Section: {section_header}")
        print(f"   Columns: {len(column_headers)}")
        print(f"   Data rows: {len(df_data)}")
    else:
        print("   (No data rows found)")
    
    print("="*80)

In [ ]:
# Test pages 91-95 (Forearm section - headers spread across row 0)
print("\n" + "="*80)
print("📊 Testing pages 91-95 (Hybrid: stream for section, lattice for data)...")
print("="*80 + "\n")

# Extract section using stream mode
page_range_91 = "91-95"
section_from_stream_91 = extract_section_from_stream(PDF_PATH, page_range_91)

# Extract table data using lattice mode
tables_91_95 = camelot.read_pdf(PDF_PATH, pages=page_range_91, flavor="lattice")

print(f"✅ Lattice mode found {len(tables_91_95)} tables in pages {page_range_91}")
if section_from_stream_91:
    print(f"✅ Stream mode found section header: '{section_from_stream_91}'")
else:
    print(f"⚠️  Stream mode did not find section header in row 0")

if len(tables_91_95) > 0:
    # Get first table
    table = tables_91_95[0]
    df = table.df
    
    print(f"\n📋 Table from pages 91-95:")
    print(f"   Raw table shape: {df.shape}")
    print(f"   Parsing accuracy: {table.accuracy:.2f}%")
    print(f"   Page: {table.page}\n")
    
    print("="*80)
    
    # Structure detection
    row0_cell0 = str(df.iloc[0, 0]).strip() if len(df) > 0 else ""
    row0_values = [str(cell).strip() for cell in df.iloc[0].tolist()] if len(df) > 0 else []
    num_filled_cells = sum(1 for v in row0_values if v and v != 'nan')
    
    if num_filled_cells > 1:
        # Case 1: Headers spread across columns (expected for pages 91-95)
        column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row0_values)]
        # Use section from stream mode if available
        section_header = section_from_stream_91 if section_from_stream_91 else "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers spread across row 0 (multiple columns)")
        if section_from_stream_91:
            print(f"✅ SECTION (from stream mode): '{section_header}'")
            print(f"   (Lattice missed it, but stream captured it in row 0)")
        else:
            print(f"⚠️  No section found - stream mode row 0 had no matching keywords")
        print(f"\n📋 Column Headers (from row 0, spread across columns):")
    elif '\n' in row0_cell0 or '\\n' in row0_cell0:
        # Case 2: Newline-separated headers
        headers_raw = row0_cell0.replace('\\n', '\n').split('\n')
        column_headers = [h.strip() for h in headers_raw if h.strip()]
        section_header = section_from_stream_91 if section_from_stream_91 else "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers in row 0, col 0 (newline-separated)")
        if section_from_stream_91:
            print(f"📚 SECTION (from stream mode): '{section_header}'")
        print(f"\n📋 Column Headers (extracted from row 0, col 0):")
    else:
        # Case 3: Section header in row 0
        section_header = row0_cell0
        print(f"📚 SECTION HEADER (from row 0): '{section_header}'")
        
        if len(df) > 1:
            row1_cell0 = str(df.iloc[1, 0]).strip()
            row1_values = [str(cell).strip() for cell in df.iloc[1].tolist()]
            num_filled_row1 = sum(1 for v in row1_values if v and v != 'nan')
            
            if '\n' in row1_cell0 or '\\n' in row1_cell0:
                headers_raw = row1_cell0.replace('\\n', '\n').split('\n')
                column_headers = [h.strip() for h in headers_raw if h.strip()]
                print(f"\n📋 COLUMN HEADERS (extracted from row 1, col 0 - newline separated):")
            elif num_filled_row1 > 1:
                column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row1_values)]
                print(f"\n📋 COLUMN HEADERS (from row 1, spread across columns):")
            else:
                column_headers = [str(h).strip() for h in df.iloc[1].tolist() if str(h).strip()]
                print(f"\n📋 COLUMN HEADERS (from row 1):")
            
            data_start_row = 2
        else:
            column_headers = []
            data_start_row = 1
    
    # Display headers
    for i, header in enumerate(column_headers):
        print(f"   {i}: {clean_text_for_display(header)}")
    
    # Data preview
    print(f"\n📊 Data Preview (first 2 rows, cleaned for display):")
    print("-"*80)
    
    if len(df) > data_start_row:
        df_data = df.iloc[data_start_row:].copy()
        
        # Apply column headers
        if len(column_headers) == df_data.shape[1]:
            df_data.columns = column_headers
        else:
            if len(column_headers) < df_data.shape[1]:
                column_headers.extend([f"Col_{i}" for i in range(len(column_headers), df_data.shape[1])])
            else:
                column_headers = column_headers[:df_data.shape[1]]
            df_data.columns = column_headers
        
        # Clean data cells for DISPLAY only
        df_display = df_data.head(2).copy()
        for col in df_display.columns:
            df_display[col] = df_display[col].apply(lambda x: clean_text_for_display(str(x)))
        
        print(df_display.to_string())
        
        print(f"\n✅ Table properly parsed:")
        print(f"   Section: {section_header}")
        print(f"   Columns: {len(column_headers)}")
        print(f"   Data rows: {len(df_data)}")
    else:
        print("   (No data rows found)")
    
    print("="*80)
else:
    print("⚠️  No tables found on pages 91-95")

---

## Test 1b: Row-by-Row Parsing (Full Output)

Extract individual rows from the table and show the complete parsed output for each row.

In [ ]:
# Import the table parser to demonstrate row-by-row parsing
from damages_parser_table import TableBasedParser
import pprint
import re

# Text cleaning function (for DISPLAY only, not for parsing)
def clean_text_for_display(text):
    """Clean text for display by replacing escape characters and normalizing whitespace."""
    if not text or text == 'nan':
        return ""
    
    text = str(text)
    text = text.replace('\\n', ' ')  # Replace literal \n
    text = text.replace('\n', ' ')   # Replace actual newlines
    text = re.sub(r'\s+', ' ', text) # Multiple spaces to single
    return text.strip()

print("📝 Demonstrating full row-by-row parsing...\n")

# Get a table with actual data
tables = camelot.read_pdf(PDF_PATH, pages="20-22", flavor="lattice")

if len(tables) > 0:
    # Get first table with data
    table = tables[0]
    df = table.df
    
    print(f"✅ Extracted table from pages 20-22")
    print(f"   Raw table shape: {df.shape}\n")
    
    print("="*80)
    
    # IMPROVED STRUCTURE DETECTION
    # Check row 0 to determine table type:
    # 1. Multiple non-empty cells across row 0 → Headers spread across columns
    # 2. Only first cell filled, has newlines → Newline-separated headers
    # 3. Only first cell filled, no newlines → Section header
    
    row0_cell0 = str(df.iloc[0, 0]).strip() if len(df) > 0 else ""
    row0_values = [str(cell).strip() for cell in df.iloc[0].tolist()] if len(df) > 0 else []
    num_filled_cells = sum(1 for v in row0_values if v and v != 'nan')
    
    if num_filled_cells > 1:
        # Case 1: Headers spread across columns (like pages 91-95)
        column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row0_values)]
        section_header = "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers spread across row 0 (multiple columns)")
        print(f"\n📋 COLUMN HEADERS (from row 0, spread across columns):")
    elif '\n' in row0_cell0 or '\\n' in row0_cell0:
        # Case 2: Newline-separated headers in first cell
        headers_raw = row0_cell0.replace('\\n', '\n').split('\n')
        column_headers = [h.strip() for h in headers_raw if h.strip()]
        section_header = "Unknown Section"
        data_start_row = 1
        print(f"📚 TABLE TYPE: Headers in row 0, col 0 (newline-separated)")
        print(f"\n📋 COLUMN HEADERS (extracted from row 0, col 0):")
    else:
        # Case 3: Section header in row 0, headers in row 1
        section_header = row0_cell0
        print(f"📚 SECTION HEADER (from row 0): '{section_header}'")
        print("   (This tells us which part of the compendium we're in)")
        
        if len(df) > 1:
            row1_cell0 = str(df.iloc[1, 0]).strip()
            row1_values = [str(cell).strip() for cell in df.iloc[1].tolist()]
            num_filled_row1 = sum(1 for v in row1_values if v and v != 'nan')
            
            if '\n' in row1_cell0 or '\\n' in row1_cell0:
                # Headers newline-separated in row 1
                headers_raw = row1_cell0.replace('\\n', '\n').split('\n')
                column_headers = [h.strip() for h in headers_raw if h.strip()]
                print(f"\n📋 COLUMN HEADERS (extracted from row 1, col 0 - newline separated):")
            elif num_filled_row1 > 1:
                # Headers spread across row 1
                column_headers = [v if v and v != 'nan' else f"Col_{i}" for i, v in enumerate(row1_values)]
                print(f"\n📋 COLUMN HEADERS (from row 1, spread across columns):")
            else:
                column_headers = [str(h).strip() for h in df.iloc[1].tolist() if str(h).strip()]
                print(f"\n📋 COLUMN HEADERS (from row 1):")
            
            data_start_row = 2
        else:
            column_headers = []
            data_start_row = 1
    
    # Display headers (cleaned for readability)
    for i, header in enumerate(column_headers):
        print(f"   {i}: {clean_text_for_display(header)}")
    
    # Create data dataframe
    df_data = df.iloc[data_start_row:].copy() if len(df) > data_start_row else df.iloc[0:0].copy()
    
    # Apply column headers
    if len(column_headers) == df_data.shape[1]:
        df_data.columns = column_headers
    else:
        if len(column_headers) < df_data.shape[1]:
            column_headers.extend([f"Col_{i}" for i in range(len(column_headers), df_data.shape[1])])
        else:
            column_headers = column_headers[:df_data.shape[1]]
        df_data.columns = column_headers
    
    print(f"\n✅ Data structure:")
    print(f"   Data shape: {df_data.shape}")
    print(f"   Number of columns: {len(column_headers)}\n")
    
    # Initialize parser
    parser = TableBasedParser(use_llm=False)  # Use rule-based for demo
    
    # Parse 3 sample rows to show full output
    print("="*80)
    print("PARSING SAMPLE ROWS (with escape characters cleaned for display)")
    print("="*80)
    
    # Get up to 3 data rows
    sample_count = min(3, len(df_data))
    
    for row_idx in range(sample_count):
        # Get RAW row data (preserve original for parsing)
        row_data_raw = df_data.iloc[row_idx].tolist()
        # Create cleaned version for display
        row_data_display = [clean_text_for_display(str(cell)) for cell in row_data_raw]
        
        actual_row_num = row_idx + data_start_row
        
        print(f"\n{'='*80}")
        print(f"ROW {actual_row_num} (data row {row_idx + 1})")
        if section_header != "Unknown Section":
            print(f"Section: {section_header}")
        print('='*80)
        
        # Show cleaned row data with column headers
        print("\n📋 Row data (cleaned for display):")
        for col_idx, (header, cell_display) in enumerate(zip(df_data.columns, row_data_display)):
            header_clean = clean_text_for_display(str(header))[:35]
            cell_val = cell_display if cell_display else "(empty)"
            if len(cell_val) > 100:
                cell_val = cell_val[:97] + "..."
            print(f"   {header_clean:35} : {cell_val}")
        
        # Parse the row using RAW data (not cleaned)
        try:
            # Map columns to expected parser fields
            row_dict = {}
            for i, header in enumerate(df_data.columns):
                header_lower = str(header).strip().lower()
                if i < len(row_data_raw):
                    value = str(row_data_raw[i]).strip()
                    
                    # Map headers to parser fields
                    if 'plaintiff' in header_lower:
                        row_dict['plaintiff'] = value
                    elif 'defendant' in header_lower:
                        row_dict['defendant'] = value
                    elif 'year' in header_lower or 'date' in header_lower:
                        row_dict['year'] = value
                    elif 'citation' in header_lower:
                        row_dict['case_citation'] = value
                    elif 'court' in header_lower:
                        row_dict['court'] = value
                    elif 'judge' in header_lower or 'justice' in header_lower:
                        row_dict['judge'] = value
                    elif any(x in header_lower for x in ['female', 'male', 'gender', 'age', 'demographic', 'sex']):
                        row_dict['demographics'] = value
                    elif 'non-pecuniary' in header_lower or ('general' in header_lower and 'damage' in header_lower):
                        row_dict['general_damages'] = value
                    elif any(x in header_lower for x in ['pecuniary', 'income', 'loss', 'special', 'other damage']):
                        row_dict['pecuniary_damages'] = value
                    elif 'injur' in header_lower or 'description' in header_lower or 'comment' in header_lower:
                        row_dict['injuries_text'] = value
            
            # Add section header as metadata
            if section_header != "Unknown Section":
                row_dict['compendium_section'] = section_header
            
            # Parse with the parser (using RAW data)
            parsed = parser.parse_row(row_dict, actual_row_num)
            
            if parsed:
                # Add section header to parsed output
                if section_header != "Unknown Section":
                    parsed['compendium_section'] = section_header
                
                print("\n✅ PARSED OUTPUT:")
                print("-" * 80)
                
                # Display key fields (cleaned for readability)
                if section_header != "Unknown Section":
                    print(f"\n📚 Compendium Section: {section_header}")
                print(f"📌 Case Name: {clean_text_for_display(str(parsed.get('case_name', 'N/A')))}")
                print(f"📅 Year: {parsed.get('year', 'N/A')}")
                print(f"⚖️  Court: {clean_text_for_display(str(parsed.get('court', 'N/A')))}")
                print(f"👨‍⚖️  Judge: {clean_text_for_display(str(parsed.get('judge', 'N/A')))}")
                
                # Extended data
                ext = parsed.get('extended_data', {})
                if ext:
                    print("\n🔍 Extended Data:")
                    
                    if ext.get('injuries'):
                        injuries_display = ext['injuries'][:3] if isinstance(ext['injuries'], list) else ext['injuries']
                        print(f"   Injuries: {injuries_display}")
                    
                    if ext.get('sex'):
                        print(f"   Sex: {ext['sex']}")
                    
                    if ext.get('age'):
                        print(f"   Age: {ext['age']}")
                    
                    if ext.get('regions'):
                        print(f"   Anatomical Regions: {ext['regions']}")
                
                # Damages breakdown
                print("\n💰 Damages:")
                if parsed.get('non_pecuniary_damages'):
                    print(f"   Non-pecuniary (General): ${parsed['non_pecuniary_damages']:,.0f}")
                if parsed.get('pecuniary_damages'):
                    print(f"   Pecuniary: ${parsed['pecuniary_damages']:,.0f}")
                if parsed.get('total_award'):
                    print(f"   Total Award: ${parsed['total_award']:,.0f}")
                
                # Full JSON output (cleaned for display, truncated for readability)
                print("\n📦 Full parsed object (first 800 chars, cleaned for display):")
                # Clean the JSON output for display
                parsed_display = json.loads(json.dumps(parsed))
                for key in parsed_display:
                    if isinstance(parsed_display[key], str):
                        parsed_display[key] = clean_text_for_display(parsed_display[key])
                full_json = json.dumps(parsed_display, indent=2)
                print(full_json[:800] + "..." if len(full_json) > 800 else full_json)
            else:
                print("\n⚠️  Row returned None (likely header or empty row)")
                
        except Exception as e:
            print(f"\n❌ Error parsing row: {e}")
            import traceback
            traceback.print_exc()
    
    print("\n" + "="*80)
    print("✅ Row-by-row parsing demonstration complete")
    if section_header != "Unknown Section":
        print(f"   Section: {section_header}")
    print(f"   Rows parsed: {sample_count}")
    print("="*80)
else:
    print("⚠️  No tables found - check PDF path and page numbers")

---

## Test 2: Injury-Focused Embedding Quality

Verify that embeddings capture injury semantics correctly (not full-text noise).

In [ ]:
# Load injury-focused embeddings
print("🔍 Loading injury-focused embeddings...\n")

model = SentenceTransformer("all-MiniLM-L6-v2")

# Load embeddings matrix and metadata
embeddings_inj = np.load("data/embeddings_inj.npy")
with open("data/ids.json") as f:
    ids = json.load(f)
with open("data/compendium_inj.json") as f:
    cases = json.load(f)

print(f"✅ Loaded {len(embeddings_inj)} injury embeddings")
print(f"   Embedding dimension: {embeddings_inj.shape[1]}")
print(f"   Total cases: {len(cases)}")

In [ ]:
# Test queries: different injury types
test_queries = [
    "C5-C6 disc herniation with chronic radicular pain to right upper extremity",
    "traumatic brain injury with persistent cognitive deficits and post-concussion syndrome",
    "lumbar facet syndrome with chronic lower back pain and limited mobility"
]

# Normalize embeddings for cosine similarity
norms = np.linalg.norm(embeddings_inj, axis=1, keepdims=True)
norms[norms == 0] = 1.0
embeddings_norm = embeddings_inj / norms

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"🔍 Query: '{query}'")
    print('='*80)
    
    # Compute query embedding
    q_emb = model.encode(query).astype("float32")
    q_norm = q_emb / np.linalg.norm(q_emb)
    
    # Cosine similarity
    sims = embeddings_norm.dot(q_norm)
    top_idx = np.argsort(-sims)[:5]
    
    print("\n📋 Top 5 matches (injury-focused semantic similarity):\n")
    for rank, idx in enumerate(top_idx, 1):
        case = cases[idx]
        print(f"{rank}. Similarity: {sims[idx]:.3f}")
        print(f"   Case: {case.get('case_name', 'Unknown')}")
        print(f"   Search text: {case.get('search_text', 'N/A')[:150]}...")
        print()

---

## Test 3: Exclusive Region Filtering

Verify that region filtering correctly includes only cases matching selected anatomical regions.

In [ ]:
# Count cases by region
from collections import Counter

print("📊 Analyzing region distribution...\n")

region_counts = Counter()
multi_region_cases = []

for case in cases:
    regions = case.get("regions") or case.get("extended_data", {}).get("regions") or []
    if isinstance(regions, str):
        regions = [regions]
    
    if len(regions) > 1:
        multi_region_cases.append(case)
    
    for r in regions:
        region_counts[str(r).strip().lower()] += 1

print("📋 Top 15 regions by case count:")
for region, count in region_counts.most_common(15):
    print(f"   {region:30} : {count:4} cases")

print(f"\n✅ Total unique regions: {len(region_counts)}")
print(f"✅ Total cases: {len(cases)}")
print(f"✅ Multi-region cases: {len(multi_region_cases)}")

In [ ]:
# Test exclusive filtering: cervical spine only
print("\n🎯 Testing EXCLUSIVE region filter (cervical spine)...\n")

selected_regions = ["cervical spine", "neck"]

filtered_cases = []
for case in cases:
    regions = case.get("regions") or case.get("extended_data", {}).get("regions") or []
    if isinstance(regions, str):
        regions = [regions]
    
    case_regions_lower = {str(r).strip().lower() for r in regions}
    selected_lower = {str(r).strip().lower() for r in selected_regions}
    
    # Exclusive: must have at least one overlap
    if case_regions_lower & selected_lower:
        filtered_cases.append(case)

print(f"Selected regions: {selected_regions}")
print(f"\n✅ Matching cases: {len(filtered_cases)} / {len(cases)} total")
print(f"   Filter ratio: {len(filtered_cases)/len(cases)*100:.1f}%")

print("\n📋 Sample filtered cases:")
for case in filtered_cases[:5]:
    print(f"   - {case['case_name'][:50]:50} | Regions: {case.get('regions', 'N/A')}")

---

## Test 4: Meta-Score Computation

Test injury overlap, age proximity, and gender match scoring.

In [ ]:
# Test injury overlap score
print("🧮 Testing injury overlap scoring...\n")

test_cases = [
    {
        "case_injuries": ["TBI", "neck strain", "PTSD"],
        "query_injuries": ["TBI", "PTSD"],
        "expected": "High overlap (2/2 match)"
    },
    {
        "case_injuries": ["cervical radiculopathy", "disc herniation"],
        "query_injuries": ["TBI", "concussion"],
        "expected": "No overlap (0/2 match)"
    },
    {
        "case_injuries": ["lumbar strain", "facet syndrome", "TBI"],
        "query_injuries": ["TBI"],
        "expected": "Exact match (1/1)"
    }
]

for i, tc in enumerate(test_cases, 1):
    score = _injury_overlap_score(tc["case_injuries"], tc["query_injuries"])
    print(f"{i}. {tc['expected']}")
    print(f"   Case injuries: {tc['case_injuries']}")
    print(f"   Query injuries: {tc['query_injuries']}")
    print(f"   Score: {score:.3f}\n")

In [ ]:
# Test age proximity score
print("\n🧮 Testing age proximity scoring...\n")

age_tests = [
    (35, 35, "Exact match"),
    (35, 38, "Within 5 years"),
    (35, 45, "Within 10 years"),
    (35, 60, "Beyond 20 years"),
    (None, 35, "Missing case age"),
]

for case_age, query_age, desc in age_tests:
    score = _age_proximity_score(case_age, query_age)
    print(f"{desc:20} : case={str(case_age):5}, query={query_age} → score={score:.2f}")

In [ ]:
# Test gender match score
print("\n🧮 Testing gender match scoring...\n")

gender_tests = [
    ("Male", "Male", "Exact match"),
    ("Female", "Male", "Mismatch"),
    (None, "Male", "Missing case gender"),
    ("male", "Male", "Case insensitive"),
]

for case_gender, query_gender, desc in gender_tests:
    score = _gender_match_score(case_gender, query_gender)
    print(f"{desc:25} : case={str(case_gender):10}, query={query_gender} → score={score:.2f}")

---

## Test 5: End-to-End Search Pipeline

Integration tests with real search queries and multiple scenarios.

In [ ]:
# Initialize app data
print("🔧 Initializing app data...\n")
model_app, cases_app, region_map = initialize_data()
print(f"✅ Loaded {len(cases_app)} cases with region map")

In [ ]:
# Scenario 1: Cervical spine injury with exclusive filter
print("\n" + "="*80)
print("🔍 SCENARIO 1: Cervical Spine Injury (Exclusive Region Filter)")
print("="*80 + "\n")

query = "C5-C6 disc herniation with chronic radicular pain to right upper extremity"
selected_regions = ["cervical spine", "neck"]

results = search_cases(
    query_text=query,
    selected_regions=selected_regions,
    cases=cases_app,
    region_map=region_map,
    model=model_app,
    gender="Male",
    age=35,
    top_n=10
)

print(f"Query: '{query}'")
print(f"Regions: {selected_regions}")
print(f"Demographics: Male, age 35")
print(f"\n✅ Results: {len(results)} cases\n")

for i, (case, inj_sim, combined) in enumerate(results[:5], 1):
    print(f"{i}. {case.get('case_name', 'Unknown')[:60]}")
    print(f"   Injury sim: {inj_sim:.3f} | Combined: {combined:.3f}")
    print(f"   Injuries: {case.get('search_text', 'N/A')[:120]}...")
    print()

In [ ]:
# Scenario 2: Multi-region injury
print("\n" + "="*80)
print("🔍 SCENARIO 2: Multi-Region Injury (Cervical + Lumbar)")
print("="*80 + "\n")

query = "cervical and lumbar disc herniations with bilateral radiculopathy"
selected_regions = ["cervical spine", "lumbar spine"]

results = search_cases(
    query_text=query,
    selected_regions=selected_regions,
    cases=cases_app,
    region_map=region_map,
    model=model_app,
    gender=None,
    age=None,
    top_n=10
)

print(f"Query: '{query}'")
print(f"Regions: {selected_regions}")
print(f"Demographics: Not specified")
print(f"\n✅ Results: {len(results)} cases\n")

for i, (case, inj_sim, combined) in enumerate(results[:5], 1):
    print(f"{i}. {case.get('case_name', 'Unknown')[:60]}")
    print(f"   Injury sim: {inj_sim:.3f} | Combined: {combined:.3f}")
    print()

In [ ]:
# Scenario 3: TBI with no region filter (all cases)
print("\n" + "="*80)
print("🔍 SCENARIO 3: Traumatic Brain Injury (No Region Filter)")
print("="*80 + "\n")

query = "traumatic brain injury with persistent cognitive deficits and headaches"
selected_regions = []  # No filter - search all

results = search_cases(
    query_text=query,
    selected_regions=selected_regions,
    cases=cases_app,
    region_map=region_map,
    model=model_app,
    gender="Female",
    age=28,
    top_n=10
)

print(f"Query: '{query}'")
print(f"Regions: All (no exclusive filter)")
print(f"Demographics: Female, age 28")
print(f"\n✅ Results: {len(results)} cases\n")

for i, (case, inj_sim, combined) in enumerate(results[:5], 1):
    ext_data = case.get('extended_data', {})
    print(f"{i}. {case.get('case_name', 'Unknown')[:60]}")
    print(f"   Injury sim: {inj_sim:.3f} | Combined: {combined:.3f}")
    print(f"   Demographics: {ext_data.get('sex', 'N/A')}, age {ext_data.get('age', 'N/A')}")
    print()

---

## Test 6: Performance Benchmarks

Measure search speed and scalability.

In [ ]:
# Benchmark search speed
print("⏱️  Running performance benchmarks...\n")

query = "cervical spine injury chronic pain"
selected_regions = ["cervical spine"]

num_runs = 10
times = []

for i in range(num_runs):
    start = time.time()
    results = search_cases(
        query_text=query,
        selected_regions=selected_regions,
        cases=cases_app,
        region_map=region_map,
        model=model_app,
        top_n=20
    )
    elapsed = time.time() - start
    times.append(elapsed)
    if i == 0:
        first_run_results = len(results)

print(f"📊 Performance Results ({num_runs} runs):")
print(f"   Corpus size: {len(cases_app):,} cases")
print(f"   Results returned: {first_run_results}")
print(f"\n   Average search time: {np.mean(times)*1000:.1f}ms")
print(f"   Min: {min(times)*1000:.1f}ms")
print(f"   Max: {max(times)*1000:.1f}ms")
print(f"   Std dev: {np.std(times)*1000:.1f}ms")

# Throughput
throughput = len(cases_app) / np.mean(times)
print(f"\n   Throughput: {throughput:,.0f} cases/second")

---

## Summary

✅ **All tests completed!**

Validation checklist:
- ✅ Camelot extracts tables correctly (lattice vs stream)
- ✅ Injury embeddings capture semantic similarity
- ✅ Exclusive region filtering works as expected
- ✅ Meta-score computation (injury overlap, age, gender)
- ✅ End-to-end search pipeline (3 scenarios)
- ✅ Performance benchmarks (speed and throughput)

**Next steps:**
1. Run `parse_and_embed.ipynb` to generate injury-focused embeddings
2. Commit precomputed artifacts (compendium_inj.json, embeddings_inj.npy, ids.json)
3. Deploy to Streamlit Cloud
4. Test live app with real queries